In [1]:
import sys
import numpy as np
from common.util import preprocess, create_contexts_target , convert_one_hot



window_size = 1
hidden_size = 5 
max_epoch = 1000

text = "you say good and i say hello."
corpus , word_to_id,id_to_word = preprocess(text)
print(corpus)

[0 1 2 3 4 1 5 6]


In [21]:
vocab_size = len(word_to_id)
contexts , target = create_contexts_target(corpus,window_size=1)
target_raw = target[...]

print(contexts)

target = convert_one_hot(target , vocab_size)
contexts = convert_one_hot(contexts , vocab_size)

print(target.shape)

print(contexts.shape)

[[0 2]
 [1 3]
 [2 4]
 [3 1]
 [4 5]
 [1 6]]
(6, 7)
(6, 2, 7)


In [4]:
import torch


In [9]:
class SimpleCBOW(torch.nn.Module):
    def __init__(self,vocab_size,hidden_size):
        super().__init__()
        # =  torch.nn.Linear()
        self.in_layer = torch.nn.Linear(vocab_size,hidden_size,bias=False)
        self.out_layer = torch.nn.Linear(hidden_size,vocab_size,bias=False)


    def forward(self ,contexts ):
        h0 = self.in_layer(contexts[:,0])
        h1 = self.in_layer(contexts[:,1])
        h = (h0 + h1) * 0.5
        score = self.out_layer(h)
        return score
        


In [22]:
contexts = torch.from_numpy(contexts).to(dtype=torch.float32)


In [28]:
target = torch.from_numpy(target_raw).to(dtype=torch.int64)
print(target)

tensor([1, 2, 3, 4, 1, 5])


In [29]:
loss_fn = torch.nn.CrossEntropyLoss()


model = SimpleCBOW(vocab_size,hidden_size)


print(model.parameters)
optimizer = torch.optim.Adam(model.parameters())
model.train()


for i in range(max_epoch):
    pred = model.forward(contexts)
    loss = loss_fn(pred,target)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if(i % 100 == 0):
        print(f"loss is {loss}")
    


#def train():


<bound method Module.parameters of SimpleCBOW(
  (in_layer): Linear(in_features=7, out_features=5, bias=False)
  (out_layer): Linear(in_features=5, out_features=7, bias=False)
)>
loss is 1.9407833814620972
loss is 1.788273811340332
loss is 1.558205008506775
loss is 1.289618730545044
loss is 1.0601271390914917
loss is 0.8724930286407471
loss is 0.7124750018119812
loss is 0.5861650109291077
loss is 0.4934132993221283
loss is 0.42614904046058655


In [31]:
print(model.in_layer.weight)


Parameter containing:
tensor([[-1.0487, -0.5184,  0.3021,  0.5584,  0.6050, -1.2696, -1.1866],
        [-0.0821,  0.6399, -1.3454,  0.7083, -0.8887, -0.1416, -1.1437],
        [-1.1492,  0.4787, -0.8763, -1.3454, -0.9806, -1.0305,  1.0655],
        [-1.1220,  1.3032, -0.7259,  1.1195, -0.7389, -0.8858,  1.1869],
        [-1.0125,  1.1471, -0.6093,  0.9583, -0.7870, -1.5254, -0.3196]],
       requires_grad=True)


In [1]:
## CBOW的改进版，pytorch 实现


import torch

vob_size = 5
embedding_dim = 2

In [5]:
input = torch.tensor([[1,2],[2,1],[1,3]])

embedding = torch.nn.Embedding(vob_size,embedding_dim)

W = embedding(input)




In [8]:
print(W)





tensor([[[-0.7367,  1.0526],
         [ 0.1018, -2.0935]],

        [[ 0.1018, -2.0935],
         [-0.7367,  1.0526]],

        [[-0.7367,  1.0526],
         [-0.0055, -1.7715]]], grad_fn=<EmbeddingBackward0>)


In [16]:
h = 0.5 * torch.sum(W,dim=-2)

print(h)

tensor([[-0.3175, -0.5204],
        [-0.3175, -0.5204],
        [-0.3711, -0.3594]], grad_fn=<MulBackward0>)


In [22]:
## 中心词，即可序测测的词

out_embed = torch.nn.Embedding(vob_size,embedding_dim)
out_put = torch.tensor([1,2,2])
target_W = out_embed(out_put)

target_W

tensor([[-0.0909, -1.2018],
        [ 1.5689,  1.0046],
        [ 1.5689,  1.0046]], grad_fn=<EmbeddingBackward0>)

In [26]:
print(target_W * h)



out = torch.sum(target_W * h ,dim= -1)

print(out)

tensor([[ 0.0288,  0.6254],
        [-0.4981, -0.5228],
        [-0.5822, -0.3611]], grad_fn=<MulBackward0>)
tensor([ 0.6543, -1.0209, -0.9433], grad_fn=<SumBackward1>)


In [57]:
### 模拟负样本

negtive_put = torch.tensor([2,1,1])
negtive_W = out_embed(negtive_put)
negtive_out = torch.sum(negtive_W * h ,dim= -1)

print(out)
print(negtive_out)
all_out = torch.cat((out,negtive_out))



print(all_out.shape)
# print(all_out[3])


tensor([ 0.6543, -1.0209, -0.9433], grad_fn=<SumBackward1>)
tensor([-1.0209,  0.6543,  0.4657], grad_fn=<SumBackward1>)
torch.Size([6])
tensor(-1.0209, grad_fn=<SelectBackward0>)


In [43]:

pos_weight = torch.ones([3])


loss_fn = torch.nn.BCEWithLogitsLoss()
y = torch.tensor([1,1,1],dtype=torch.float32)


print(out)
loss = loss_fn(all_out,y)


loss.backward()



tensor([ 0.6543, -1.0209, -0.9433], grad_fn=<SumBackward1>)
